### Dataset

In [390]:
import pandas as pd
import warnings
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings('ignore')

import h2o
from h2o.automl import H2OAutoML

In [391]:
# Evito que ciertas columnas se transformen a notacion cientifica en las predicciones
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

In [392]:
columns = [
    # 'Open_time',
    'Open',
    'High',
    'Low',
    # 'Close',
    'Number of trades',
    # 'Close_BTCUSDT',
    # 'Volume_BTCUSDT',
    # 'Number_of_trades_BTCUSDT',
    # 'Close_ETHUSDT',
    # 'Volume_ETHUSDT',
    # 'Number_of_trades_ETHUSDT',
    # 'Close_BNBUSDT',
    # 'Volume_BNBUSDT',
    # 'Number_of_trades_BNBUSDT',
    # 'SMA_20',
    'EMA_20',
    'Upper_Band',
    'Middle_Band',
    # 'Lower_Band',
    'RSI',
    # 'buy_1000x_high_coinbase',
    # 'sell_1000x_high_coinbase',
    # 'total_trades_coinbase',	
    # 'Tweets_Utilizados',
    # 'Tweets_Utilizados_coin',
    # 'Tweets_Utilizados_referentes',
    # 'Tweets_Utilizados_whale_alert',
    'Buy_1000x_high',
    'sell_1000x_high',
    # 'total_trades_binance'
]


# VOLVER A AGRAGAR LAS COLUMNAS DE ACA ABAJO PARA LA PRIMER VERSION PERFORMANTE DE H2O
# Number_of_trades_BTCUSDT	0.6588976	0.0071163	0.0029513
# Volume_BTCUSDT	0.5628256	0.0060787	0.0025209
# Volume_ETHUSDT	0.5505871	0.0059465	0.0024661
# Volume_BNBUSDT	0.2992841	0.0032324	0.0013405
# total_trades_coinbase	0.2962261	0.0031993	0.0013268
# Tweets_Utilizados	0.2395923	0.0025877	0.0010732
# Tweets_Utilizados_coin	0.1547555	0.0016714	0.0006932
# Tweets_Utilizados_whale_alert	0.1284685	0.0013875	0.0005754

### Armado y entrenamiento de un clasificador a partir de los datos originales

In [393]:
complete_dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv') 
classifier_dataset = complete_dataset[columns]
classifier_dataset['Tendencia'] = complete_dataset['Tendencia']

clasifier_validation = classifier_dataset[-10:]
classifier_dataset = classifier_dataset[:-10]

In [394]:
# PARA NORMALIZACION
scaler = MinMaxScaler() #StandardScaler()
features = classifier_dataset[columns]
features_scaled = scaler.fit_transform(features)

classifier_dataset_scaled = pd.DataFrame(features_scaled, columns=columns)
classifier_dataset_scaled['Tendencia'] = classifier_dataset['Tendencia'].values

In [395]:
# PARA NORMALIZACION
features_validation = clasifier_validation[columns]
features_validation_scaled = scaler.transform(features_validation)
clasifier_validation_scaled = pd.DataFrame(features_validation_scaled, columns=columns)
clasifier_validation_scaled['Tendencia'] = clasifier_validation['Tendencia'].values

In [396]:
display(classifier_dataset_scaled.tail())

,Open,High,Low,Number of trades,EMA_20,Upper_Band,Middle_Band,RSI,Buy_1000x_high,sell_1000x_high,Tendencia
941,0.07,0.06,0.06,0.02,0.08,0.09,0.08,0.26,0.12,0.12,Lateral
942,0.06,0.06,0.06,0.02,0.08,0.09,0.08,0.24,0.16,0.19,Lateral
943,0.06,0.06,0.06,0.02,0.08,0.09,0.08,0.25,0.13,0.09,Lateral
944,0.06,0.06,0.06,0.01,0.08,0.08,0.07,0.24,0.08,0.09,Lateral
945,0.06,0.06,0.06,0.02,0.08,0.07,0.07,0.21,0.13,0.10,Bajista


In [397]:
classifier_dataset.shape

(946, 11)

In [398]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,11 days 19 hours 23 mins
H2O_cluster_timezone:,America/Argentina/Buenos_Aires
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,2 months and 14 days
H2O_cluster_name:,H2O_from_python_mmarchetta_uvjkm7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.482 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [399]:
# h2o_df = h2o.H2OFrame(classifier_dataset)
h2o_df = h2o.H2OFrame(classifier_dataset_scaled) # PARA NORMALIZACION

# aml = H2OAutoML(nfolds=5, keep_cross_validation_predictions=True, seed=1234) <= para la primer version performante de h2o
# aml = H2OAutoML(nfolds=5, keep_cross_validation_predictions=True, stopping_rounds=5, exploitation_ratio=0.1, seed=12345) <= para la segunda version performante de h2o
# aml = H2OAutoML(nfolds=10, keep_cross_validation_predictions=True, stopping_rounds=5, exploitation_ratio=0.1, seed=12345) <= para la tercer version performante de h2o
# aml = H2OAutoML(nfolds=5, keep_cross_validation_predictions=True, stopping_rounds=5, exploitation_ratio=0.4, seed=12345, max_runtime_secs= 60 * 60 * 4) <= para la cuarta version performante de h2o

aml = H2OAutoML(nfolds=5, keep_cross_validation_predictions=True, stopping_rounds=7, exploitation_ratio=0.5, seed=3579, max_runtime_secs= 60 * 60 * 4)

aml.train(y='Tendencia', training_frame=h2o_df)

Parse progress: |

████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████Failed polling AutoML progress log: Unexpected HTTP error: ('Connection aborted.', BadStatusLine('GET /99/AutoML/AutoML_20_20240528_140031@@Tendencia?verbosity=warn HTTP/1.1\r\n'))
| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),255/273
# GBM base models (used / total),40/44
# XGBoost base models (used / total),39/42
# DeepLearning base models (used / total),173/184
# GLM base models (used / total),1/1
# DRF base models (used / total),2/2
Metalearner algorithm,GBM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [400]:
leaderboard = aml.leaderboard
print(leaderboard)

model_id                                                    mean_per_class_error    logloss      rmse       mse
StackedEnsemble_AllModels_5_AutoML_20_20240528_140031                   0.197011   0.496638  0.387624  0.150253
DeepLearning_grid_1_AutoML_20_20240528_140031_model_6                   0.202196   0.5075    0.394969  0.156001
StackedEnsemble_BestOfFamily_6_AutoML_20_20240528_140031                0.205252   0.505602  0.406018  0.16485
DeepLearning_grid_1_AutoML_20_20240528_140031_model_39                  0.205485   0.601604  0.392942  0.154403
StackedEnsemble_AllModels_6_AutoML_20_20240528_140031                   0.206223   0.469983  0.388199  0.150698
DeepLearning_grid_1_AutoML_20_20240528_140031_model_76                  0.212939   0.606054  0.402622  0.162104
StackedEnsemble_AllModels_4_AutoML_20_20240528_140031                   0.213818   0.552423  0.438078  0.191912
DeepLearning_grid_1_AutoML_20_20240528_140031_model_60                  0.21671    0.585802  0.40403   0.

### Guardado de los mejores 5 modelos

In [401]:
top_models = aml.leaderboard.as_data_frame()['model_id'].tolist()[:5]
for i, model_id in enumerate(top_models):
    model = h2o.get_model(model_id)
    file_name = f"modelo_{i}.h2o"
    h2o.save_model(model=model, path=f"automl_classifier", filename=file_name)


### Cargo los mejores 5 modelos

In [402]:
stack_models = []
for i in range(5):
    model_path = f"automl_classifier/modelo_{i}.h2o"
    model = h2o.load_model(model_path)
    stack_models.append(model)

### Hago las predicciones

In [403]:
stack_models[0]

key,value
Stacking strategy,cross_validation
Number of base models (used / total),255/273
# GBM base models (used / total),40/44
# XGBoost base models (used / total),39/42
# DeepLearning base models (used / total),173/184
# GLM base models (used / total),1/1
# DRF base models (used / total),2/2
Metalearner algorithm,GBM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [411]:
h2o_validation_df = h2o.H2OFrame(clasifier_validation_scaled) # PARA NORMALIZACION
# h2o_validation_df = h2o.H2OFrame(clasifier_validation[columns])
predictions = stack_models[0].predict(h2o_validation_df)
display(predictions)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,Alcista,Bajista,Lateral
Bajista,0.00604322,0.963479,0.0304778
Alcista,0.980527,0.00677358,0.0126997
Alcista,0.964944,0.0041513,0.0309044
Lateral,0.117939,0.377392,0.504669
Bajista,0.00465019,0.82979,0.16556
Alcista,0.908354,0.00503929,0.086607
Bajista,0.00384898,0.953211,0.0429399
Bajista,0.00576217,0.605396,0.388842
Lateral,0.00816666,0.184001,0.807832
Lateral,0.0123938,0.0138328,0.973773


##### Datos generados por auto ml con prophet:

In [405]:
# auto_ml_prophet_df = pd.read_csv('auto_timeseries_models_prophet/predicciones.csv')
# h2o_prophet_df = h2o.H2OFrame(auto_ml_prophet_df[columns])
# auto_mp_prophet_predictions = stack_models[2].predict(h2o_prophet_df)

# display(auto_mp_prophet_predictions)

#PARA NORMALIZACION
auto_ml_prophet_df = pd.read_csv('auto_timeseries_models_prophet/predicciones.csv')
features_prophet = auto_ml_prophet_df[columns]
features_prophet_scaled = scaler.transform(features_prophet)
auto_ml_prophet_df_scaled = pd.DataFrame(features_prophet_scaled, columns=columns)
h2o_prophet_df = h2o.H2OFrame(auto_ml_prophet_df_scaled)
auto_mp_prophet_predictions = stack_models[0].predict(h2o_prophet_df)

display(auto_mp_prophet_predictions)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,Alcista,Bajista,Lateral
Bajista,0.00330025,0.985771,0.0109292
Bajista,0.00321243,0.985905,0.0108821
Bajista,0.00331519,0.985105,0.0115795
Bajista,0.00330404,0.985925,0.0107708
Bajista,0.00329468,0.984619,0.0120863
Bajista,0.00337325,0.984294,0.0123323
Bajista,0.00321074,0.986647,0.0101418
Bajista,0.00318919,0.980024,0.0167868
Bajista,0.00324274,0.982901,0.0138559
Bajista,0.00512424,0.974045,0.0208303


##### Datos generados por auto ml con stats:

In [406]:
# auto_ml_stats_df = pd.read_csv('auto_timeseries_models/predicciones.csv')
# h2o_stats_df = h2o.H2OFrame(auto_ml_stats_df[columns])
# auto_mp_stats_predictions = stack_models[2].predict(h2o_stats_df)

# display(auto_mp_stats_predictions)

# PARA NORMALIZACION
auto_ml_stats_df = pd.read_csv('auto_timeseries_models/predicciones.csv')
features_stats = auto_ml_stats_df[columns]
features_stats_scaled = scaler.transform(features_stats)
auto_ml_stats_df_scaled = pd.DataFrame(features_stats_scaled, columns=columns)
h2o_stats_df = h2o.H2OFrame(auto_ml_stats_df_scaled)
auto_mp_stats_predictions = stack_models[0].predict(h2o_stats_df)

display(auto_mp_stats_predictions)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,Alcista,Bajista,Lateral
Lateral,0.00997172,0.0192209,0.970807
Lateral,0.00924694,0.455317,0.535436
Lateral,0.018551,0.452923,0.528526
Lateral,0.00656168,0.482432,0.511007
Bajista,0.0067689,0.818884,0.174347
Bajista,0.00518234,0.888009,0.106809
Bajista,0.00502585,0.97255,0.0224244
Bajista,0.00303828,0.980795,0.0161671
Bajista,0.0047872,0.943811,0.0514023
Bajista,0.00719263,0.887648,0.105159


##### Datos generados por auto ml con modelos clasicos:

In [407]:
# auto_ml_df = pd.read_csv('h2o_models/predicciones.csv')
# h2o_mp_df = h2o.H2OFrame(auto_ml_df[columns])
# auto_mp_predictions = stack_models[2].predict(h2o_mp_df)

# display(auto_mp_predictions)

#PARA NORAMLIZACION
auto_ml_df = pd.read_csv('h2o_models/predicciones.csv')
features_ml = auto_ml_df[columns]
features_ml_scaled = scaler.transform(features_ml)
auto_ml_df_scaled = pd.DataFrame(features_ml_scaled, columns=columns)
h2o_mp_df = h2o.H2OFrame(auto_ml_df_scaled)
auto_mp_predictions = stack_models[0].predict(h2o_mp_df)

display(auto_mp_predictions)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,Alcista,Bajista,Lateral
Bajista,0.00576503,0.968038,0.0261974
Bajista,0.00560621,0.582272,0.412122
Bajista,0.00605272,0.940507,0.0534405
Bajista,0.00705714,0.934017,0.0589263
Bajista,0.00697241,0.892442,0.100586
Bajista,0.0063091,0.898763,0.0949283
Bajista,0.0060603,0.936952,0.0569873
Bajista,0.00667737,0.936417,0.0569053
Bajista,0.00657785,0.945248,0.0481743
Bajista,0.00495906,0.946108,0.0489325


##### Datos generados por auto ml con modelos clasicos:

In [408]:
# skforecast_df = pd.read_csv('skforecast/predicciones.csv')
# skforecast_df = h2o.H2OFrame(skforecast_df)
# skforecast_predictions = stack_models[2].predict(skforecast_df)

# display(skforecast_predictions)

# PARA NORMALIZACION
skforecast_df = pd.read_csv('skforecast/predicciones.csv')
features_skforecast = skforecast_df[columns]
features_skforecast_scaled = scaler.transform(features_skforecast)
skforecast_df_scaled = pd.DataFrame(features_skforecast_scaled, columns=columns)
h2o_skforecast_df = h2o.H2OFrame(skforecast_df_scaled)
skforecast_predictions = stack_models[0].predict(h2o_skforecast_df)

display(skforecast_predictions)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,Alcista,Bajista,Lateral
Bajista,0.012087,0.643966,0.343947
Bajista,0.00316932,0.981944,0.0148863
Bajista,0.00318393,0.986468,0.0103477
Bajista,0.00352136,0.988182,0.00829708
Bajista,0.00367931,0.987608,0.00871266
Bajista,0.0044858,0.983817,0.0116971
Bajista,0.00334297,0.989258,0.00739896
Bajista,0.00538332,0.983711,0.0109053
Bajista,0.00393195,0.989186,0.00688228
Bajista,0.00413419,0.98863,0.00723626


In [409]:
# stack_models[0].explain(h2o_validation_df)

In [410]:
# from matplotlib import pyplot as plt


# ensemble_model = stack_models[0]

# # Obtener los modelos base del ensemble
# base_models_ids = ensemble_model.base_models

# # Visualizar la importancia de características para cada modelo base
# for model_id in base_models_ids:
#     base_model = h2o.get_model(model_id)
#     try:
#         # Generar y mostrar el gráfico de importancia de características
#         base_model.varimp_plot()
#         plt.show()
#     except:
#         print(f"No se puede generar la importancia de características para el modelo {model_id} de tipo {base_model.algo}")
